<a href="https://colab.research.google.com/github/tanaymukherjee/Deep-Learning-with-PyTorch/blob/master/01_Introduction_to_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to PyTorch

## Import packages and libraries

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

## Table of Contents

1. Basic autograd example 1
2. Basic autograd example 2
3. Loading data from numpy
4. Input pipline
5. Input pipline for custom dataset
6. Pretrained model
7. Save and load model 

## 1. Basic autograd example 1

### Create tensors

In [3]:
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

### Build a computational graph

In [4]:
y = w * x + b    # y = 2 * x + 3

### Compute gradients

In [5]:
y.backward()

### Print out the gradients

In [6]:
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


## 2. Basic autograd example 2

### Create tensors of shape (10, 3) and (10, 2)

In [7]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

### Build a fully connected layer

In [8]:
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.5365,  0.0821, -0.4799],
        [-0.0822, -0.4136,  0.2844]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.3979, -0.0299], requires_grad=True)


### Build loss function and optimizer

In [9]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

### Forward pass

In [10]:
pred = linear(x)

### Compute Loss

In [11]:
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  1.7813125848770142


### Backward pass

In [12]:
loss.backward()

### Print out the gradients

In [13]:
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[-0.2016, -0.1495, -1.2145],
        [ 0.2762, -0.2103,  0.6792]])
dL/db:  tensor([-0.2657,  0.7464])


### 1-step gradient descent

In [14]:
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

### Print out the loss after 1-step gradient descent

In [15]:
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.7540979385375977


## 3. Loading data from numpy

### Create a numpy array

In [16]:
x = np.array([[1, 2], [3, 4]])

### Convert the numpy array to a torch tensor

In [17]:
y = torch.from_numpy(x)

### Convert the torch tensor to a numpy array

In [18]:
z = y.numpy()

## 4. Input pipeline

### Download and construct CIFAR-10 dataset

In [19]:
train_dataset = torchvision.datasets.CIFAR10(root='../../data/', train=True, transform=transforms.ToTensor(), download=True)

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


### Fetch one data pair (read data from disk)

In [20]:
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


### Data loader (this provides queues and threads in a very simple way)

In [21]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

### When iteration starts, queue and thread start to load data from files

In [22]:
data_iter = iter(train_loader)

### Mini-batch images and labels

In [23]:
images, labels = data_iter.next()

### Actual usage of the data loader is as below

In [24]:
for images, labels in train_loader:
    # Training code should be written here.
    pass

## 5. Input pipeline for custom dataset

### You should build your custom dataset as below

In [28]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

### You can then use the prebuilt data loader

In [ ]:
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

## 6. Pretrained model 

### Download and load the pretrained ResNet-18

In [29]:
resnet = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


### If you want to finetune only the top layer of the model, set as below

In [30]:
for param in resnet.parameters():
    param.requires_grad = False

### Replace the top layer for finetuning

In [31]:
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

### Forward pass

In [32]:
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

torch.Size([64, 100])


## 7. Save and load the model

### Save and load the entire model

In [34]:
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

### Save and load only the model parameters (recommended)

In [35]:
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>